In [1]:
#ISTALLING LIBRARIES

In [2]:
!pip install checklist

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install -U nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [5]:
!python3 -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 145.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/sgupta32/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
#Importing Libraries
import checklist
from checklist.editor import Editor
from checklist.test_types import MFT, INV, DIR
from checklist.perturb import Perturb
import spacy
from pattern.en import sentiment
from checklist.expect import Expect
import pandas as pd
import numpy as np
from checklist.pred_wrapper import PredictorWrapper

In [8]:
#Read the train_seen dataset
df = pd.read_csv("/scratch/sgupta32/Learn-To-Adapt/train_seen.csv") 

In [9]:
#Adding typos
ret1 = Perturb.perturb(df['Text'], Perturb.add_typos)
ret1.data[:5]

[["i'd like to hear some trivia about florida",
  "i'd like to hear some trivia about flroida"],
 ["please tell me the bill's minimum payment",
  "please tellm e the bill's minimum payment"],
 ['who gave you life', 'wh ogave you life'],
 ['at what time will my flight begin boarding',
  'at whatt ime will my flight begin boarding'],
 ['my card is lost, i would like to report it',
  'my car dis lost, i would like to report it']]

In [10]:
#Adding negations
nlp = spacy.load('en_core_web_sm')
pdata = list(nlp.pipe(df['Text']))

In [12]:
#Run it twice
ret2 = Perturb.perturb(pdata, Perturb.add_negation)
ret2.data[:5]

[["i'd like to hear some trivia about florida",
  "i wouldn't like to hear some trivia about florida"],
 ["please tell me the bill's minimum payment",
  "please don't tell me the bill's minimum payment"],
 ['who gave you life', "who didn't give you life"],
 ['at what time will my flight begin boarding',
  "at what time won't my flight begin boarding"],
 ['my card is lost, i would like to report it',
  "my card is lost, i wouldn't like to report it"]]

In [13]:
#Changing the names
ret3 = Perturb.perturb(pdata, Perturb.change_names)
ret3.data[:5]

[['has my reservation for john at bardenay been confirmed',
  'has my reservation for Antonio at bardenay been confirmed',
  'has my reservation for Ethan at bardenay been confirmed',
  'has my reservation for Christian at bardenay been confirmed',
  'has my reservation for Nicholas at bardenay been confirmed',
  'has my reservation for Mason at bardenay been confirmed',
  'has my reservation for Noah at bardenay been confirmed',
  'has my reservation for Jose at bardenay been confirmed',
  'has my reservation for Samuel at bardenay been confirmed',
  'has my reservation for Richard at bardenay been confirmed',
  'has my reservation for Henry at bardenay been confirmed'],
 ['bella needs to be called',
  'Rachel needs to be called',
  'Alexis needs to be called',
  'Melanie needs to be called',
  'Rebecca needs to be called',
  'Angela needs to be called',
  'Karen needs to be called',
  'Brittany needs to be called',
  'Riley needs to be called',
  'Alicia needs to be called',
  'Carol

In [14]:
############Minimum Functionality Test (MFT)############

In [15]:
editor = Editor()

In [16]:
#Initializing positive and negative words
pos = ['good', 'enjoyable', 'exciting', 'excellent', 'amazing', 'great', 'engaging', 'like']
neg = ['bad', 'terrible', 'awful', 'horrible', 'lost']

In [17]:
#Adding columns in the df
df['pos'] = df['Text'].str.count('|'.join(pos))
df['neg'] = df['Text'].str.count('|'.join(neg))

#Conditions depending on the count of positive words and negative words
conditions = [
(df['pos'] > df['neg']),
(df['neg'] > df['pos']),
(df['neg'] == df['pos'])
]

#Available choices
choices = [
'positive',
'negative',
'neutral'
]

# Assigning the appropriate choice based on the conditions
df['overall'] = np.select(conditions, choices, default = '')

In [18]:
ret = editor.template('I am good.', pos=pos, labels=0, save=True, nsamples=100)
for i in range(0, 12600):
    # Checking if the 'overall' value for the current row is 'positive'
    if df['overall'][i] == 'positive':
        # Generating templates for the positive text
        ret += editor.template(df['Text'][i], pos=pos, labels=0, save=True, nsamples=100)
    # Checking if the 'overall' value for the current row is 'negative'
    if df['overall'][i] == 'negative':
        # Generating templates for the negative text
        ret += editor.template(df['Text'][i], neg=neg, labels=1, save=True, nsamples=100)

In [19]:
# The code creates an MFT (Minimal Functionality Test) object named 'test' using the provided data and labels.
# The 'name' parameter is set to 'Simple negation'.
# The 'capability' parameter is set to 'Negation'.
# The 'description' parameter is set to 'Very simple negations.'
test = MFT(ret.data, labels=ret.labels, name='Simple negation',
           capability='Negation', description='Very simple negations.')

In [20]:
# The code above creates an MFT (Minimal Functionality Test) object named 'test' using the values from the 'ret' dictionary.
# The 'name' parameter is set to 'Simple negation'.
# The 'capability' parameter is set to 'Negation'.
# The 'description' parameter is set to 'Very simple negations.'
test = MFT(**ret, name='Simple negation',
           capability='Negation', description='Very simple negations.')

In [21]:
def predict_proba(inputs):
    # Calculate the predicted probabilities for the positive class
    p1 = np.array([(sentiment(x)[0] + 1)/2. for x in inputs]).reshape(-1, 1)
    # Calculate the predicted probabilities for the negative class
    p0 = 1 - p1
    # Return the predicted probabilities as a two-column array
    return np.hstack((p0, p1))

In [22]:
predict_proba(['good', 'bad'])

array([[0.15, 0.85],
       [0.85, 0.15]])

In [23]:
wrapped_pp = PredictorWrapper.wrap_softmax(predict_proba)

In [24]:
wrapped_pp(['good'])

(array([1]), array([[0.15, 0.85]]))

In [25]:
test.run(wrapped_pp)

Predicting 814 examples


In [26]:
test.summary()

Test cases:      814
Fails (rate):    425 (52.2%)

Example fails:
0.8 what's a good restaurant nearby
----
0.8 i would like to place an order for more checkbooks to me mailed to me
----
0.8 good morning, ai
----


In [27]:
############INVARIANCE TESTS############

In [28]:
#typos
test = INV(**ret1)

In [29]:
test.run(wrapped_pp)
test.summary()

Predicting 25200 examples
Test cases:      12600
Fails (rate):    290 (2.3%)

Example fails:
0.2 i am very confused about the block that seems to have been placed on my account why is it there
0.6 i am very confusde about the block that seems to have been placed on my account why is it there

----
0.7 what do you enjoy doing in your leisure time
0.5 what do youe njoy doing in your leisure time

----
0.4 um, i'm not actually sure
0.8 um, i'm not actualyl sure

----


In [30]:
#negations
monotonic_decreasing = Expect.monotonic(label=1, increasing=False, tolerance=0.1)

In [31]:
test = DIR(**ret2, expect = monotonic_decreasing)

In [32]:
test.run(wrapped_pp)
test.summary()

Predicting 23384 examples
Test cases:      11692
After filtering: 11635 (99.5%)
Fails (rate):    56 (0.5%)

Example fails:
0.2 my card got scratched badly, can you tell my bank to send a replacement
0.7 my card got scratched badly, can't you tell my bank to send a replacement

----
0.4 that is not true, it's false
0.5 that is not true, it's not false

----
0.2 that's wrong
0.6 that's not wrong

----


In [33]:
#name change
test = INV(**ret3)

In [34]:
test.run(wrapped_pp)
test.summary()

Predicting 1958 examples
Test cases:      178
Fails (rate):    0 (0.0%)


In [35]:
############Creating a new Train Dataset with noise############

In [36]:
#Add negations to df
for i in range(0, 12600):
    df['Text'][i] = ret2.data[i][1]

/tmp/ipykernel_3540803/2180183994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'][i] = ret2.data[i][1]


IndexError: list index out of range

In [ ]:
#Adding typos to new df
ret1 = Perturb.perturb(df['Text'], Perturb.add_typos)
ret1.data[:5]

In [ ]:
#Add typos to df
for i in range(0, 12600):
    df['Text'][i] = ret1.data[i][1]

In [37]:
df.to_csv(path_or_buf='/scratch/sgupta32/Learn-To-Adapt/data/ver1/train_seen.csv', index=False, columns=["Text", "Label"])